## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
os.path.join

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,63.03,81,99,4.00,overcast clouds
1,1,Mahebourg,MU,-20.4081,57.7000,68.29,73,40,6.91,scattered clouds
2,2,Busselton,AU,-33.6500,115.3333,59.02,90,100,10.74,light rain
3,3,Isangel,VU,-19.5500,169.2667,78.91,73,75,4.61,broken clouds
4,4,Drinovci,BA,43.3550,17.3539,67.08,79,4,3.98,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you'd like for your trip?"))
max_temp = float(input("What is the maximum temperature you'd like for your trip?"))
min_temp
max_temp

85.0

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
temp_preferred_cities_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Isangel,VU,-19.5500,169.2667,78.91,73,75,4.61,broken clouds
5,5,Norman Wells,CA,65.2820,-126.8329,75.24,22,40,11.50,scattered clouds
8,8,Lorengau,PG,-2.0226,147.2712,84.15,70,65,5.44,broken clouds
16,16,Rabaul,PG,-4.1967,152.1721,84.54,68,60,6.69,broken clouds
36,36,Aklavik,CA,68.2191,-135.0107,77.88,26,100,12.33,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
689,689,San Benedetto Del Tronto,IT,42.9456,13.8816,78.04,51,10,5.55,clear sky
694,694,Cumana,VE,10.4635,-64.1775,78.87,84,74,5.75,broken clouds
698,698,San Pedro,PH,14.3500,121.0167,84.31,72,75,6.91,broken clouds
701,701,Nouakchott,MR,18.0858,-15.9785,78.73,79,7,14.79,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
print (len(temp_preferred_cities_df))
temp_preferred_cities_df.count()

184


Unnamed: 0             184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_preferred_cities_df.dropna(axis = "index", how = "any")


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Isangel,VU,78.91,broken clouds,-19.5500,169.2667,
5,Norman Wells,CA,75.24,scattered clouds,65.2820,-126.8329,
8,Lorengau,PG,84.15,broken clouds,-2.0226,147.2712,
16,Rabaul,PG,84.54,broken clouds,-4.1967,152.1721,
36,Aklavik,CA,77.88,overcast clouds,68.2191,-135.0107,
42,Avera,US,83.71,clear sky,33.1940,-82.5271,
50,Butaritari,KI,81.73,light rain,3.0707,172.7902,
51,Papetoai,PF,80.19,scattered clouds,-17.4961,-149.8729,
52,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
55,Acajutla,SV,78.35,moderate rain,13.5928,-89.8275,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
    
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis = "index", how = "any")
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Isangel,VU,78.91,broken clouds,-19.5500,169.2667,Tanna Lodge
5,Norman Wells,CA,75.24,scattered clouds,65.2820,-126.8329,Heritage Hotel
8,Lorengau,PG,84.15,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
16,Rabaul,PG,84.54,broken clouds,-4.1967,152.1721,Rabaul Hotel
36,Aklavik,CA,77.88,overcast clouds,68.2191,-135.0107,Aklavik Inn


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{City}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0, 32.0), zoom_level= 1.5)
markers = gmaps.marker_layer (locations, info_box_content= hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))